In [1]:
# Compute LAI RADAR map using Geosys APIs
# Get the last LAI value for multiple fields
# Get LAI RADAR values from AF at the last optical image date and at the last radar image date
# Compute the radar offset
# Get the new LAI map using the offset

# Author: @mal
# Version: 1.0
# Date: 21/11/2022

In [2]:
## load libraries & API connection & load data & parameters

from geosyspy import Geosys
import os
from dotenv import load_dotenv
import datetime as dt
from dateutil.relativedelta import relativedelta
import logging
from geosyspy.constants import Collection, Region, Env
import os.path as pa
import sys
import os
from os import listdir
from os.path import isfile, join
import requests
import requests.exceptions as rex
import datetime as dt
import pandas as pd
import geopandas as gpd
import numpy as np
import getpass
import json

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# read .env file
load_dotenv('C:/Users/mal/OneDrive - GEOSYS/Methodology/Python_script/GeosysPy-develop/ppus.env')

API_CLIENT_ID = os.getenv('API_CLIENT_ID')
API_CLIENT_SECRET = os.getenv('API_CLIENT_SECRET')
API_USERNAME = os.getenv('API_USERNAME')
API_PASSWORD = os.getenv('API_PASSWORD')

client = Geosys(API_CLIENT_ID, API_CLIENT_SECRET, API_USERNAME, API_PASSWORD, Env.PREPROD, Region.NA)

access_token = client.token['access_token']
access_token

# Load field id
input_file = pd.read_csv('C:/Users/mal/OneDrive - GEOSYS/Methodology/Python_script/GeosysPy-develop/2020_RADAR_demo_PPUS_script_5fields.csv',sep=';')

# Print the data table.
print(input_file.head(10))

# Parameters
maps_type=['INSEASON_LAI']
start_date = '2020-03-01'
end_date = '2020-04-28'

INFO:root:Authenticated


  sfd_externalid  start date    end date
0        aj7qpn3  01/01/2020  30/06/2020
1        mp2w57e  01/01/2020  30/06/2020
2        5np36j1  01/01/2020  30/06/2020
3        y3bv51g  01/01/2020  30/06/2020
4        bg7pjl7  01/01/2020  30/06/2020


In [3]:
## Get LAI coverage (no recalibrated)

response=[]
Image_date=[]
Image_date_null=[]

for i in range(len(input_file)):
               
    get_coverage_url = "http://api-pp.geosys-na.net/field-level-maps/v4/season-fields/"+input_file["sfd_externalid"][i]+'/catalog-imagery?Image.Date=$between:'+start_date+'|'+end_date+'&Maps.Type=INSEASON_NDVI&coverageType=CLEAR&Image.isrecalibrated=false&$fields=image.id,seasonField.id,coverageType,image.sensor,image.date,Image.isrecalibrated'
    headers={'Authorization':'Bearer '+access_token,
                                            'Accept':'application/json','Content-Type': 'application/json'
                                            }

    print(get_coverage_url)
    
    response = requests.request("GET", get_coverage_url, headers=headers)
    #print(response.json)
    
    if response.status_code == 401:
        print('no access token, please recall the authentication API')

    res_coverage=response.json()
    #print(len(res_coverage))
    #print(res_coverage)

    if (res_coverage == []):
        #case if the API call retreive any images that coverage the field
        data_res={}
        data_res["sensor_type"]='null'
        Image_date_null.append(data_res)
    else:
        for j in range(len(res_coverage)):
            data_res={}
            data_res["season_field_id"]=input_file["sfd_externalid"][i]
            data_res["image_date"]=res_coverage[j]['image']['date']
            data_res["image_id"]=res_coverage[j]['image']['id']
            data_res["sensor_type"]=res_coverage[j]['image']['sensor']
            Image_date.append(data_res)
            

http://api-pp.geosys-na.net/field-level-maps/v4/season-fields/aj7qpn3/catalog-imagery?Image.Date=$between:2020-03-01|2020-04-28&Maps.Type=INSEASON_NDVI&coverageType=CLEAR&Image.isrecalibrated=false&$fields=image.id,seasonField.id,coverageType,image.sensor,image.date,Image.isrecalibrated
http://api-pp.geosys-na.net/field-level-maps/v4/season-fields/mp2w57e/catalog-imagery?Image.Date=$between:2020-03-01|2020-04-28&Maps.Type=INSEASON_NDVI&coverageType=CLEAR&Image.isrecalibrated=false&$fields=image.id,seasonField.id,coverageType,image.sensor,image.date,Image.isrecalibrated
http://api-pp.geosys-na.net/field-level-maps/v4/season-fields/5np36j1/catalog-imagery?Image.Date=$between:2020-03-01|2020-04-28&Maps.Type=INSEASON_NDVI&coverageType=CLEAR&Image.isrecalibrated=false&$fields=image.id,seasonField.id,coverageType,image.sensor,image.date,Image.isrecalibrated
http://api-pp.geosys-na.net/field-level-maps/v4/season-fields/y3bv51g/catalog-imagery?Image.Date=$between:2020-03-01|2020-04-28&Maps.Typ

In [4]:
# Initialisation & create a coverage table
path='C:/Users/mal/OneDrive - GEOSYS/Methodology/Python_script/GeosysPy-develop'

output_file = pd.DataFrame(columns=[
                                    'Season_field_id',
                                    'image_date',
                                    'image_id',
                                    'Sensor_type'
                                        ])


output_file=[]

for i in range(len(Image_date)):
    output_file.append({'Season_field_id':Image_date[i]['season_field_id'],
                                   'Image_date':Image_date[i]['image_date'],
                                    'Image_id':Image_date[i]['image_id'],
                                   'Sensor_type':Image_date[i]['sensor_type']
                                    })

output_file=pd.DataFrame(output_file)

print(output_file.head(len(Image_date)))
# Enregistrement des données en csv
output_file.to_csv(pa.join(path,'Image_date_LAI.csv'),
                               sep=';',
                               index=False) 

   Season_field_id  Image_date                                     Image_id  \
0          aj7qpn3  2020-04-26  IKc73hpUQ6o6MWx9U3twc7GU2d5NrDVCtmdn6u1KWUm   
1          aj7qpn3  2020-04-26  IKc73hpUQ6o6MWx9U3twc7GU2d5NrDVAluY5cnQKpBE   
2          aj7qpn3  2020-04-23  IKc73hpUQ6o4mXCyJFndOv569lzbyEIoHXpjFL9ceiS   
3          aj7qpn3  2020-04-23  IKc73hpUQ6o4mXCyJFndOv569lzbyEInDbmsVHr7o3g   
4          aj7qpn3  2020-04-16  IKc73hpUQ6o16dF6yPiPVJaBJwXa8oO0kD3IV63HrC0   
5          aj7qpn3  2020-04-08  IKc73hpUQ6nwtgb0kafxFUDxjMOjEIK1tKi9MHYKv0W   
6          aj7qpn3  2020-03-27  IKc73hpUQ6nqaqWWqz11bBDp3K6gCumATIznR9pMMlA   
7          aj7qpn3  2020-03-27  IKc73hpUQ6nqaqWWqz11bBDp3K6gCum9PMwwh6WrW6O   
8          aj7qpn3  2020-03-24  IKc73hpUQ6np0rFLc5MVGINGcImhzCeqb4AESqX1tfI   
9          aj7qpn3  2020-03-24  IKc73hpUQ6np0rFLc5MVGINGcImhzCepB9R68m8MlSG   
10         aj7qpn3  2020-03-22  IKc73hpUQ6nnxtO6q0qz7ONZcgAhLkMqdOwgAfThpRs   
11         aj7qpn3  2020-03-22  IKc73hpUQ6nnxtO6q0qz

In [5]:
# Get LAI values

response=[]
res_BRF=[]
get_BRF_url=[]
LAI_values=[]

for m in range(len(maps_type)):
    for i in range(len(Image_date)):
        print(output_file["Image_date"][i])
        if output_file["Image_date"][i] =='no image':
            pass
        else:
            get_BRF_url = "http://api-pp.geosys-na.net/field-level-maps/v4/season-fields/"+output_file["Season_field_id"][i]+"/coverage/"+output_file["Image_id"][i]+"/base-reference-map/"+maps_type[m]
            print(get_BRF_url)
            headers={'Authorization':'Bearer '+access_token,
                                                'Accept':'application/json','Content-Type': 'application/json'
                                                }

            response = requests.request("GET", get_BRF_url, headers=headers)
            res_BRF=response.json()
            #print(res_BRF["legend"]["stat"]["mean"])
            mean_LAI_value = res_BRF["legend"]["stat"]["mean"]
            LAI_res={}
            LAI_res["season_field_id"]=res_BRF["seasonField"]["id"]
            LAI_res["image_date"]=Image_date[i]['image_date']
            LAI_res["image_id"]=Image_date[i]['image_id']
            LAI_res["sensor_type"]=Image_date[i]['sensor_type']
            LAI_res["lai_mean"]=res_BRF["legend"]["stat"]["mean"]
            LAI_values.append(LAI_res)
            print(LAI_res)

2020-04-26
http://api-pp.geosys-na.net/field-level-maps/v4/season-fields/aj7qpn3/coverage/IKc73hpUQ6o6MWx9U3twc7GU2d5NrDVCtmdn6u1KWUm/base-reference-map/INSEASON_LAI
{'season_field_id': 'aj7qpn3', 'image_date': '2020-04-26', 'image_id': 'IKc73hpUQ6o6MWx9U3twc7GU2d5NrDVCtmdn6u1KWUm', 'sensor_type': 'SENTINEL_2', 'lai_mean': 2.3170855}
2020-04-26
http://api-pp.geosys-na.net/field-level-maps/v4/season-fields/aj7qpn3/coverage/IKc73hpUQ6o6MWx9U3twc7GU2d5NrDVAluY5cnQKpBE/base-reference-map/INSEASON_LAI
{'season_field_id': 'aj7qpn3', 'image_date': '2020-04-26', 'image_id': 'IKc73hpUQ6o6MWx9U3twc7GU2d5NrDVAluY5cnQKpBE', 'sensor_type': 'SENTINEL_2', 'lai_mean': 2.319519}
2020-04-23
http://api-pp.geosys-na.net/field-level-maps/v4/season-fields/aj7qpn3/coverage/IKc73hpUQ6o4mXCyJFndOv569lzbyEIoHXpjFL9ceiS/base-reference-map/INSEASON_LAI
{'season_field_id': 'aj7qpn3', 'image_date': '2020-04-23', 'image_id': 'IKc73hpUQ6o4mXCyJFndOv569lzbyEIoHXpjFL9ceiS', 'sensor_type': 'SENTINEL_2', 'lai_mean': 2.13

{'season_field_id': 'mp2w57e', 'image_date': '2020-03-24', 'image_id': 'IKc73hpUQ6np0rFLc5MVGINGcImhzCeqb4AESqX1tfI', 'sensor_type': 'SENTINEL_2', 'lai_mean': 0.6580933}
2020-03-24
http://api-pp.geosys-na.net/field-level-maps/v4/season-fields/mp2w57e/coverage/IKc73hpUQ6np0rFLc5MVGINGcImhzCepB9R68m8MlSG/base-reference-map/INSEASON_LAI
{'season_field_id': 'mp2w57e', 'image_date': '2020-03-24', 'image_id': 'IKc73hpUQ6np0rFLc5MVGINGcImhzCepB9R68m8MlSG', 'sensor_type': 'SENTINEL_2', 'lai_mean': 0.6549701}
2020-03-22
http://api-pp.geosys-na.net/field-level-maps/v4/season-fields/mp2w57e/coverage/IKc73hpUQ6nnxtO6q0qz7ONZcgAhLkMqdOwgAfThpRs/base-reference-map/INSEASON_LAI
{'season_field_id': 'mp2w57e', 'image_date': '2020-03-22', 'image_id': 'IKc73hpUQ6nnxtO6q0qz7ONZcgAhLkMqdOwgAfThpRs', 'sensor_type': 'SENTINEL_2', 'lai_mean': 0.7859814}
2020-03-22
http://api-pp.geosys-na.net/field-level-maps/v4/season-fields/mp2w57e/coverage/IKc73hpUQ6nnxtO6q0qz7ONZcgAhLkMpDUDXqb52hEq/base-reference-map/INSEA

{'season_field_id': 'y3bv51g', 'image_date': '2020-03-17', 'image_id': 'IKc73hpUQ6nlKyRz71OkYTDQP3lBJyRhbrtCfcD3klI', 'sensor_type': 'SENTINEL_2', 'lai_mean': 0.50349724}
2020-03-12
http://api-pp.geosys-na.net/field-level-maps/v4/season-fields/y3bv51g/coverage/IKc73hpUQ6nii154860odWhlFUwIdAUwbamncoJcTHI/base-reference-map/INSEASON_LAI
{'season_field_id': 'y3bv51g', 'image_date': '2020-03-12', 'image_id': 'IKc73hpUQ6nii154860odWhlFUwIdAUwbamncoJcTHI', 'sensor_type': 'SENTINEL_2', 'lai_mean': 0.32187024}
2020-03-12
http://api-pp.geosys-na.net/field-level-maps/v4/season-fields/y3bv51g/coverage/IKc73hpUQ6nii154860odWhlFUwIdAUvXejwsl17ccW/base-reference-map/INSEASON_LAI
{'season_field_id': 'y3bv51g', 'image_date': '2020-03-12', 'image_id': 'IKc73hpUQ6nii154860odWhlFUwIdAUvXejwsl17ccW', 'sensor_type': 'SENTINEL_2', 'lai_mean': 0.32512492}
2020-04-26
http://api-pp.geosys-na.net/field-level-maps/v4/season-fields/bg7pjl7/coverage/IKc73hpUQ6o6MWx9U3twc7GU2d5NrDV9LzoxIj1fgyC/base-reference-map/IN

In [6]:
# Create LAI table

output_file = pd.DataFrame(columns=[
                                    'Season_field_id',
                                    'image_date',
                                    'image_id',
                                    'Sensor_type',
                                    'LAI_mean'
                                        ])


output_file=[]

for w in range(len(LAI_values)):
    output_file.append({'Season_field_id':LAI_values[w]['season_field_id'],
                                   'Image_date':LAI_values[w]['image_date'],
                                   'Sensor_type':LAI_values[w]['sensor_type'],
                                    'Image_id':LAI_values[w]['image_id'],
                                    'LAI_mean':LAI_values[w]['lai_mean']
                                    })

output_file=pd.DataFrame(output_file)

print(output_file.head(len(LAI_values)))
# Enregistrement des données en csv
output_file.to_csv(pa.join(path,'LAI_values_radar.csv'),
                               sep=';',
                               index=False) 

   Season_field_id  Image_date Sensor_type  \
0          aj7qpn3  2020-04-26  SENTINEL_2   
1          aj7qpn3  2020-04-26  SENTINEL_2   
2          aj7qpn3  2020-04-23  SENTINEL_2   
3          aj7qpn3  2020-04-23  SENTINEL_2   
4          aj7qpn3  2020-04-16  SENTINEL_2   
5          aj7qpn3  2020-04-08  SENTINEL_2   
6          aj7qpn3  2020-03-27  SENTINEL_2   
7          aj7qpn3  2020-03-27  SENTINEL_2   
8          aj7qpn3  2020-03-24  SENTINEL_2   
9          aj7qpn3  2020-03-24  SENTINEL_2   
10         aj7qpn3  2020-03-22  SENTINEL_2   
11         aj7qpn3  2020-03-22  SENTINEL_2   
12         aj7qpn3  2020-03-17  SENTINEL_2   
13         aj7qpn3  2020-03-17  SENTINEL_2   
14         aj7qpn3  2020-03-12  SENTINEL_2   
15         aj7qpn3  2020-03-12  SENTINEL_2   
16         mp2w57e  2020-04-26  SENTINEL_2   
17         mp2w57e  2020-04-26  SENTINEL_2   
18         mp2w57e  2020-04-23  SENTINEL_2   
19         mp2w57e  2020-04-23  SENTINEL_2   
20         mp2w57e  2020-04-16  SE

In [17]:
# Get metrics from Analytics Fabric & compute & download new LAI map

schema_id = "LAI_RADAR"
start_date_radar = dt.datetime.strptime(start_date, "%Y-%m-%d")
end_date_radar = dt.datetime.strptime(end_date, "%Y-%m-%d")
response=[]
res_BRF=[]
get_BRF_url=[]
LAI_values_final=[]
    
for i in range(len(input_file)):
    seasonfield = input_file["sfd_externalid"][i]    
    lai_radar = client.get_metrics_by_seasonfieldid(seasonfield, schema_id, start_date_radar, end_date_radar)
    lai_radar
    last_radar_value = lai_radar["Values.LAI"][len(lai_radar)-1]
    last_radar_date = end_date_radar
    last_optical_image_date = dt.datetime.strptime(LAI_values[0]['image_date'], "%Y-%m-%d")
    dif_date = (end_date_radar-last_optical_image_date)
    optical_date_radar = client.get_metrics_by_seasonfieldid(seasonfield, schema_id, start_date_radar, last_optical_image_date)
    optical_date_radar_value = optical_date_radar["Values.LAI"][len(optical_date_radar)-1]
    offset = (float(last_radar_value) - float(optical_date_radar_value))
    offset_radar = str(offset)
    LAI_values2 = output_file[output_file['Season_field_id'].isin([seasonfield])]
    final_lai = LAI_values2.iat[0,4] + offset
    
    #radar rules
    if ((LAI_values2.iat[0,4] < 0.5) 
        or (LAI_values2.iat[0,4] > 3) 
        or (last_radar_value < 0) 
        or (last_radar_value > 3) 
        or (optical_date_radar_value < 0) 
        or (optical_date_radar_value > 3) 
        or (offset < 0)
        or (dif_date < dt.timedelta(days=2)) 
        or (dif_date > dt.timedelta(days=30))
       ):       
        rules_results = 'N'
    else:
        rules_results = 'Y'
    
    print('seasonfield_id=',seasonfield)
    print('last_radar_value=',last_radar_value)
    print('optical_date_radar_value=',optical_date_radar_value)
    print('offset_radar=',offset_radar)    
    print('last_optical_lai=',LAI_values2.iat[0,4])
    print('final_lai=',final_lai)
    print('image_date=',LAI_values2.iat[0,1])
    print('rules=',rules_results)
    
    get_BRF_url = "http://api-pp.geosys-na.net/field-level-maps/v4/season-fields/"+seasonfield+"/coverage/"+LAI_values2.iat[0,3]+"/base-reference-map/"+maps_type[m]+"?offset="+offset_radar
    
    headers={'Authorization':'Bearer '+access_token,
                                                'Accept':'application/json','Content-Type': 'application/json'
                                                }

    response = requests.request("GET", get_BRF_url, headers=headers)
    res_BRF=response.json()
    LAI_res={}
    LAI_res["season_field_id"]=seasonfield
    LAI_res["image_date"]=Image_date[i]['image_date']
    LAI_res["image_id"]=Image_date[i]['image_id']
    LAI_res["sensor_type"]=Image_date[i]['sensor_type']
    LAI_res["lai_mean"]=res_BRF["legend"]["stat"]["mean"]
    LAI_res["last_radar_value"]=last_radar_value
    LAI_res["optical_date_radar_value"]=optical_date_radar_value
    LAI_res["offset"]=offset_radar
    LAI_res["final_lai"]=final_lai
    LAI_res["rules"]=rules_results
    LAI_values_final.append(LAI_res)
    print(LAI_res)
            #search the url in the aims to save the image in GEOTIFF format :

    tiff_url=res_BRF['_links']['image:image/tiff+zip']
    #print(tiff_url)
    response_tiff = requests.request("GET", tiff_url, headers=headers)
    tiff_path=path+'/TIFF_radar'+'/'+maps_type[m]+'_'+seasonfield+'_'+LAI_values2.iat[0,1]+'_'+LAI_values2.iat[0,2]+'_tiff'+'.zip'
    with open(tiff_path, 'wb') as f:
            f.write(response_tiff.content)

INFO:root:Calling APIs for metrics


https://api-pp.geosys-na.net/analytics/metrics?%24limit=9999&Timestamp=$between:2020-03-01|2020-04-28&$filter=Entity.ExternalTypedIds.Contains("SeasonField:aj7qpn3@LEGACY_ID_NA")&$filter=Schema.Id==LAI_RADAR


INFO:root:Calling APIs for metrics


https://api-pp.geosys-na.net/analytics/metrics?%24limit=9999&Timestamp=$between:2020-03-01|2020-04-26&$filter=Entity.ExternalTypedIds.Contains("SeasonField:aj7qpn3@LEGACY_ID_NA")&$filter=Schema.Id==LAI_RADAR
seasonfield_id= aj7qpn3
last_radar_value= 1.8519866177105073
optical_date_radar_value= 1.7228669555192369
offset_radar= 0.12911966219127047
last_optical_lai= 2.3170855
final_lai= 2.4462051621912706
image_date= 2020-04-26
rules= Y
{'season_field_id': 'aj7qpn3', 'image_date': '2020-04-26', 'image_id': 'IKc73hpUQ6o6MWx9U3twc7GU2d5NrDVCtmdn6u1KWUm', 'sensor_type': 'SENTINEL_2', 'lai_mean': 2.4461904, 'last_radar_value': 1.8519866177105073, 'optical_date_radar_value': 1.7228669555192369, 'offset': '0.12911966219127047', 'final_lai': 2.4462051621912706, 'rules': 'Y'}


INFO:root:Calling APIs for metrics


...

https://api-pp.geosys-na.net/analytics/metrics?%24limit=9999&Timestamp=$between:2020-03-01|2020-04-26&$filter=Entity.ExternalTypedIds.Contains("SeasonField:y3bv51g@LEGACY_ID_NA")&$filter=Schema.Id==LAI_RADAR
seasonfield_id= y3bv51g
last_radar_value= 1.0577453660359872
optical_date_radar_value= 1.0447724707752488
offset_radar= 0.012972895260738415
last_optical_lai= 0.94161505
final_lai= 0.9545879452607384
image_date= 2020-04-26
rules= Y
{'season_field_id': 'y3bv51g', 'image_date': '2020-04-23', 'image_id': 'IKc73hpUQ6o4mXCyJFndOv569lzbyEInDbmsVHr7o3g', 'sensor_type': 'SENTINEL_2', 'lai_mean': 0.95421153, 'last_radar_value': 1.0577453660359872, 'optical_date_radar_value': 1.0447724707752488, 'offset': '0.012972895260738415', 'final_lai': 0.9545879452607384, 'rules': 'Y'}


INFO:root:Calling APIs for metrics


https://api-pp.geosys-na.net/analytics/metrics?%24limit=9999&Timestamp=$between:2020-03-01|2020-04-28&$filter=Entity.ExternalTypedIds.Contains("SeasonField:bg7pjl7@LEGACY_ID_NA")&$filter=Schema.Id==LAI_RADAR


INFO:root:Calling APIs for metrics


https://api-pp.geosys-na.net/analytics/metrics?%24limit=9999&Timestamp=$between:2020-03-01|2020-04-26&$filter=Entity.ExternalTypedIds.Contains("SeasonField:bg7pjl7@LEGACY_ID_NA")&$filter=Schema.Id==LAI_RADAR
seasonfield_id= bg7pjl7
last_radar_value= 2.42559298681146
optical_date_radar_value= 2.374550347209078
offset_radar= 0.05104263960238198
last_optical_lai= 2.7452114
final_lai= 2.796254039602382
image_date= 2020-04-26
rules= Y
{'season_field_id': 'bg7pjl7', 'image_date': '2020-04-16', 'image_id': 'IKc73hpUQ6o16dF6yPiPVJaBJwXa8oO0kD3IV63HrC0', 'sensor_type': 'SENTINEL_2', 'lai_mean': 2.7962968, 'last_radar_value': 2.42559298681146, 'optical_date_radar_value': 2.374550347209078, 'offset': '0.05104263960238198', 'final_lai': 2.796254039602382, 'rules': 'Y'}


In [18]:
# Create LAI RADAR table

output_file = pd.DataFrame(columns=[
                                    'Season_field_id',
                                    'image_date',
                                    'image_id',
                                    'Sensor_type',
                                    'Optical_LAI_mean',
                                    'last_radar_value',
                                    'optical_date_radar_value',
                                    'offset_radar',
                                    'final_lai',
                                    'rules',
                                        ])


output_file=[]

for z in range(len(LAI_values_final)):
    output_file.append({'Season_field_id':LAI_values_final[z]['season_field_id'],
                                   'Image_date':LAI_values_final[z]['image_date'],
                                   'Sensor_type':LAI_values_final[z]['sensor_type'],
                                    'Image_id':LAI_values_final[z]['image_id'],
                                    'LAI_mean':LAI_values_final[z]['lai_mean'],
                                    'Optical_LAI_mean':LAI_values_final[z]['lai_mean'],
                                    'last_radar_value':LAI_values_final[z]['last_radar_value'],
                                    'optical_date_radar_value':LAI_values_final[z]['optical_date_radar_value'],
                                    'offset_radar':LAI_values_final[z]['offset'],
                                    'final_lai':LAI_values_final[z]['final_lai'],
                                    'rules':LAI_values_final[z]['rules'],
                                    })

output_file=pd.DataFrame(output_file)

print(output_file.head(len(LAI_res)))

# Enregistrement des données en csv
output_file.to_csv(pa.join(path,'LAI_values_radar_final.csv'),
                               sep=';',
                               index=False)

  Season_field_id  Image_date Sensor_type  \
0         aj7qpn3  2020-04-26  SENTINEL_2   
1         mp2w57e  2020-04-26  SENTINEL_2   
2         5np36j1  2020-04-23  SENTINEL_2   
3         y3bv51g  2020-04-23  SENTINEL_2   
4         bg7pjl7  2020-04-16  SENTINEL_2   

                                      Image_id  LAI_mean  Optical_LAI_mean  \
0  IKc73hpUQ6o6MWx9U3twc7GU2d5NrDVCtmdn6u1KWUm  2.446190          2.446190   
1  IKc73hpUQ6o6MWx9U3twc7GU2d5NrDVAluY5cnQKpBE  1.450014          1.450014   
2  IKc73hpUQ6o4mXCyJFndOv569lzbyEIoHXpjFL9ceiS  1.550846          1.550846   
3  IKc73hpUQ6o4mXCyJFndOv569lzbyEInDbmsVHr7o3g  0.954212          0.954212   
4  IKc73hpUQ6o16dF6yPiPVJaBJwXa8oO0kD3IV63HrC0  2.796297          2.796297   

   last_radar_value  optical_date_radar_value          offset_radar  \
0          1.851987                  1.722867   0.12911966219127047   
1          1.253173                  1.195777   0.05739578737105511   
2          1.497623                  1.374978  